# Constants:

In [1]:
SEQ_LEN = 240   # how many past candles to use to predict
CANDLES_SHIFT = 5 # how many candles to shift between sequences
NAME = "shift5_seq240_ov40th004"
VALIDATION_PCT = 0.2

DF initialisation:

In [4]:
import pandas as pd
main_df = pd.read_csv("data/classified/HistoricalDataClassified_2016_2023_300_ov40_th004.csv")
main_df

,timestamp,BTC_close,BTC_low,BTC_high,BTC_volume,BTC_average,BTC_HLPercent,target
0,1.467332e+09,667.00,667.00,671.00,2764.20,668.04,0.005961,2
1,1.467332e+09,670.10,670.10,671.99,8873.00,671.99,0.002813,2
2,1.467332e+09,667.00,667.00,672.00,204.46,671.90,0.007440,2
3,1.467332e+09,667.00,667.00,672.00,0.00,671.90,0.007440,2
4,1.467333e+09,667.00,667.00,672.00,0.00,671.90,0.007440,2
...,...,...,...,...,...,...,...,...
710115,1.680366e+09,28423.92,28416.67,28440.89,4032.00,28424.22,0.000852,0
710116,1.680366e+09,28402.64,28401.90,28426.65,1226.36,28409.15,0.000871,0
710117,1.680366e+09,28418.48,28403.11,28418.48,4559.18,28406.68,0.000541,0
710118,1.680367e+09,28412.62,28407.49,28422.84,388.83,28413.01,0.000540,0


# Functions:

sequence split

In [5]:
def splitDf(df):
    
    res = []
    print("")
    print("splitDf")
    while len(df) >= SEQ_LEN + len(df.columns) -1:
        first = df.head(SEQ_LEN + len(df.columns) -1).copy()
        first.index = np.arange(0, len(first))
        res.append(first)
        df = df.tail(len(df) - CANDLES_SHIFT)
        df.index = np.arange(0, len(df))

    print("-done")
    print("")
    return res

In [6]:
def splitDf_new(df):
    
    res = []
    print("")
    print("splitDf")
    while len(df) >= SEQ_LEN:
        first = df.head(SEQ_LEN).copy()
        first.index = np.arange(0, len(first))
        res.append(first)
        df = df.tail(len(df) - CANDLES_SHIFT)
        df.index = np.arange(0, len(df))

    print("-done")
    print("")
    return res

balance

In [7]:
def balance(dfs):
    
    buys = []
    sells = []
    holds = []
    for df in dfs:
        if df.at[len(df)-1, 'target'] == 0:
            sells.append(df)
        elif df.at[len(df)-1, 'target'] == 1:
            buys.append(df)
        else:
          holds.append(df)

    print("before balancing:")
    print("buys:", len(buys), ", sells:", len(sells), ", holds:", len(holds))

    smallest = min(len(buys), len(sells), len(holds))
    buys = buys[:smallest]
    sells = sells[:smallest]
    holds = holds[:smallest]

    dfsBalanced = buys+sells+holds
    return dfsBalanced

preprocessing

In [8]:
from sklearn import preprocessing
from tqdm import tqdm

def preprocess(dfs):
    
    for df in dfs:
        for col in df.columns:
            if col != "target":
                df[col] = df[col].pct_change()
                df.dropna(inplace=True)
                df[col] = preprocessing.scale(df[col].values)
                df.index = np.arange(0, len(df))

    return dfs

In [9]:
import numpy as np

def new_preprocess(df):

    for col in ["BTC_close", "BTC_low", "BTC_high", "BTC_average"]:
        #print(col)
        df[col] = np.log(df[col])
        df[col] = df[col].pct_change()
        df.dropna(inplace=True)
        #mean = np.mean(df[col])
        mean = 0.00000068
        #std = np.std(df[col])
        std = 0.00028
        #print("mean:", mean, ", std:", std)
        df[col] = (df[col] - mean) / std
    
    
    df["BTC_volume"] = df["BTC_volume"].replace(0, 1)
    df["BTC_volume"] = np.log(df["BTC_volume"])
    #df["BTC_volume"] = df["BTC_volume"].pct_change()   # taking the pct change somehow makes it worse
    #df.dropna(inplace=True)                            # taking the pct change somehow makes it worse
    #mean = np.mean(df["BTC_volume"])
    mean = 9.3
    #std = np.std(df["BTC_volume"])
    std = 2.82
    #print("mean:", mean, ", std:", std)
    df["BTC_volume"] = (df["BTC_volume"] - mean) / std


    #mean = np.mean(df["BTC_HLPercent"])
    mean = 0.0027
    #std = np.std(df["BTC_HLPercent"])
    std = 0.0032
    #print("mean:", mean, ", std:", std)
    df["BTC_HLPercent"] = (df["BTC_HLPercent"] - mean) / std


    return df

sequences

In [10]:
def buildSequences(dfs):
    
    sequences = []
    for df in dfs:
        if(len(df) == SEQ_LEN):
            label = df.at[SEQ_LEN-1, 'target']
            df = df.iloc[:, :-1]
            dfArray = df.values.tolist()
            sequences.append([np.array(dfArray), label])
    
    return sequences

X y split

In [11]:
def split(seqWithTarget):

    X = []
    y = []
    for seq, target in seqWithTarget:
        X.append(seq)
        y.append(target)

    return np.array(X),np.array(y)

# DF manipulation, build training sets:

In [12]:
# new preprocessing before splitting

df_preprocessed = new_preprocess(main_df)
df_preprocessed.drop(["timestamp"], axis=1, inplace=True)
print(df_preprocessed)

        BTC_close   BTC_low  BTC_high  BTC_volume  BTC_average  BTC_HLPercent  \
4       -0.002429 -0.002429 -0.002429   -3.297872    -0.002429       1.481399   
5       -0.002429 -0.002429 -0.002429   -3.297872    -0.002429       1.481399   
6        0.376210  0.252769 -1.637556   -0.842769    -2.596816       0.410914   
7        2.083405  2.206908  0.816119   -0.129185     1.235813      -0.378027   
8        0.816119 -0.084351 -0.002429   -0.864965    -0.084317      -0.331455   
...           ...       ...       ...         ...          ...            ...   
710115  -0.205381  0.253832  0.002469   -0.353894     0.335073      -0.577628   
710116  -0.263258 -0.183495 -0.176833   -0.775955    -0.187120      -0.571668   
710117   0.191756  0.012409 -0.102535   -0.310320    -0.032711      -0.674736   
710118  -0.074250  0.051276  0.050999   -1.183283     0.075172      -0.674982   
710119   0.113388 -0.281215 -0.011618   -1.268209    -0.075859      -0.433308   

        target  
4         

split into dfs with SEQ_LEN rows

In [13]:
import numpy as np
#splittedDfs = splitDf(main_df)
splittedDfs = splitDf_new(main_df)


splitDf
-done



In [14]:
print("num sequences:", len(splittedDfs))

num sequences: 141976


seperate training and validation

In [15]:
dfsTraining = splittedDfs[:(int(len(splittedDfs) * (1-VALIDATION_PCT)))].copy()
dfsValidation = splittedDfs[(int(len(splittedDfs) * (1-VALIDATION_PCT))):].copy()

shuffle

In [16]:
import random

random.shuffle(dfsTraining)
random.shuffle(dfsValidation)

balance buys/sells/holds

In [17]:
dfsTrainingBalanced = balance(dfsTraining)
dfsValidationBalanced = balance(dfsValidation) # balance(dfsValidation) , validation data does not have to be balanced   

before balancing:
buys: 14012 , sells: 13158 , holds: 86410
before balancing:
buys: 2587 , sells: 2674 , holds: 23135


shuffle

In [18]:
random.shuffle(dfsTrainingBalanced)
random.shuffle(dfsValidationBalanced)

preprocessing

In [43]:
#dfsTrainingPreprocessed = preprocess(dfsTrainingBalanced)
#dfsValidationPreprocessed = preprocess(dfsValidationBalanced)

build sequences

In [19]:
#sequencesTraining = buildSequences(dfsTrainingPreprocessed)
#sequencesValidation = buildSequences(dfsValidationPreprocessed)
sequencesTraining = buildSequences(dfsTrainingBalanced)
sequencesValidation = buildSequences(dfsValidationBalanced)

shuffle

In [20]:
random.shuffle(sequencesTraining)
random.shuffle(sequencesValidation)

split sequence from label

In [21]:
train_x, train_y = split(sequencesTraining)
validation_x, validation_y = split(sequencesValidation)

In [22]:
print(type(train_x))
print(train_x.shape)
print(type(validation_x))
print(validation_x.shape)

<class 'numpy.ndarray'>
(39474, 240, 6)
<class 'numpy.ndarray'>
(7761, 240, 6)


In [23]:
import pickle

with open(f'data/pkl/{NAME}_train_x.pkl', 'wb') as file:
    pickle.dump(train_x, file)
with open(f'data/pkl/{NAME}_train_y.pkl', 'wb') as file:
    pickle.dump(train_y, file)
with open(f'data/pkl/{NAME}_validation_x.pkl', 'wb') as file:
    pickle.dump(validation_x, file)
with open(f'data/pkl/{NAME}_validation_y.pkl', 'wb') as file:
    pickle.dump(validation_y, file)

In [2]:
import pickle

with open(f"data/pkl/{NAME}/{NAME}_train_x.pkl", "rb") as file:
    train_x = pickle.load(file)
with open(f"data/pkl/{NAME}/{NAME}_train_y.pkl", "rb") as file:
    train_y = pickle.load(file)
with open(f"data/pkl/{NAME}/{NAME}_validation_x.pkl", "rb") as file:
    validation_x = pickle.load(file)
with open(f"data/pkl/{NAME}/{NAME}_validation_y.pkl", "rb") as file:
    validation_y = pickle.load(file)

# Model:

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


hyper parameters bounds

In [8]:
EPOCHS = 400

batchSize = 96
layers = 2
nodes = 256
denseNodes = 128

dropOut = 0.92 #0.88
rec_dropout = 0
l1l2_reg = 0 #1e-5#1e-3

learningRate = 0.00001
decay = 0

hp_suffix = f"bs({batchSize})_layers({layers})_noded({nodes})_dNodes({denseNodes})_do({int(dropOut*100)}%)_recdo({rec_dropout})_l1l2({l1l2_reg})_lr({learningRate})_decay({decay})"

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os
from tqdm.keras import TqdmCallback
import datetime
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import regularizers
import json
# model
model = Sequential()

for _ in range(layers-1):
  model.add(Bidirectional(LSTM(nodes, 
                               activation="tanh", 
                               recurrent_activation = 'sigmoid', 
                               recurrent_dropout = rec_dropout, 
                               unroll = False, 
                               use_bias = True, 
                               input_shape=(train_x.shape[1:]), 
                               return_sequences=True,
                               kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                               #bias_regularizer=regularizers.l2(l1l2_reg),
                               activity_regularizer=regularizers.l2(l1l2_reg)
                               )))
  model.add(Dropout(dropOut))
  #model.add(BatchNormalization())

model.add(Bidirectional(LSTM(nodes, 
                             activation="tanh", 
                             recurrent_activation = 'sigmoid', 
                             recurrent_dropout = rec_dropout, 
                             unroll = False, 
                             use_bias = True, 
                             input_shape=(train_x.shape[1:]),
                             kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                             #bias_regularizer=regularizers.l2(l1l2_reg),
                             activity_regularizer=regularizers.l2(l1l2_reg)
                             )))
model.add(Dropout(dropOut))
#model.add(BatchNormalization())

model.add(Dense(denseNodes, 
                activation="relu",
                kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                #bias_regularizer=regularizers.l2(l1l2_reg), 
                activity_regularizer=regularizers.l2(l1l2_reg)))
model.add(Dropout(dropOut))

model.add(Dense(3, activation="softmax"))



# opt  
opt = tf.keras.optimizers.Adam(lr=learningRate) # decay?


model.compile(loss="sparse_categorical_crossentropy",
              optimizer = opt,
              metrics=["accuracy"])


filename = "bi_" + NAME + "-{epoch:02d}"
filepath = f"models/{filename}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1) # saves every epoch

# Prepare TensorBoard callback
log_dir = f"logs/fit/bi_{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# store hyper-parameters
# Path to the JSON file
file_path = "models/hps/hps.json"
# Load the JSON file into a dictionary
with open(file_path, "r") as json_file:
    data = json.load(json_file)
# Add a new key-value pair to the dictionary
data[f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")] = hp_suffix
# Write the updated dictionary back to the file
with open(file_path, "w") as json_file:
    json.dump(data, json_file, indent=4)




# train
history = model.fit(
  train_x, train_y,
  batch_size = batchSize,
  epochs = EPOCHS,
  validation_data=(validation_x, validation_y),
  callbacks=[checkpoint, TqdmCallback(verbose=0), tensorboard_callback])
  #callbacks=[checkpoint])

c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)




Epoch 1/400
411/412 [============================>.] - ETA: 0s - loss: 1.3258 - accuracy: 0.3363

412/412 [==============================] - 29s 60ms/step - loss: 1.3257 - accuracy: 0.3363 - val_loss: 1.0904 - val_accuracy: 0.4811
Epoch 2/400
411/412 [============================>.] - ETA: 0s - loss: 1.2734 - accuracy: 0.3441

412/412 [==============================] - 24s 58ms/step - loss: 1.2734 - accuracy: 0.3441 - val_loss: 1.0847 - val_accuracy: 0.5050
Epoch 3/400
411/412 [============================>.] - ETA: 0s - loss: 1.2375 - accuracy: 0.3516

412/412 [==============================] - 24s 58ms/step - loss: 1.2375 - accuracy: 0.3516 - val_loss: 1.0812 - val_accuracy: 0.5096
Epoch 4/400
411/412 [============================>.] - ETA: 0s - loss: 1.2222 - accuracy: 0.3522

...

412/412 [==============================] - 25s 60ms/step - loss: 0.9357 - accuracy: 0.5833 - val_loss: 0.8957 - val_accuracy: 0.6136
Epoch 173/400
411/412 [============================>.] - ETA: 0s - loss: 0.9361 - accuracy: 0.5835

412/412 [==============================] - 25s 61ms/step - loss: 0.9363 - accuracy: 0.5834 - val_loss: 0.9041 - val_accuracy: 0.5959
Epoch 174/400
411/412 [============================>.] - ETA: 0s - loss: 0.9391 - accuracy: 0.5841

412/412 [==============================] - 24s 58ms/step - loss: 0.9392 - accuracy: 0.5841 - val_loss: 0.8988 - val_accuracy: 0.6086
Epoch 175/400
412/412 [==============================] - ETA: 0s - loss: 0.9351 - accuracy: 0.5834

412/412 [==============================] - 25s 59ms/step - loss: 0.9351 - accuracy: 0.5834 - val_loss: 0.9015 - val_accuracy: 0.6006
Epoch 176/400
157/412 [==========>...................] - ETA: 14s - loss: 0.9356 - accuracy: 0.5809

KeyboardInterrupt: 